<a href="https://colab.research.google.com/github/Alias23/IO-PRACTICAS/blob/main/IO_PRACTICA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from pulp import LpProblem, LpVariable, LpMaximize, LpMinimize, lpSum, value, LpStatus

# ----------------------------
# Problema 3: Plantaciones
# ----------------------------

# Datos
total_hect = 50
agua_disponible = 100_000  # litros
min_maiz_kg = 5120         # kg de maíz mínimo

# Rendimientos y parámetros por cultivo
#        producción (kg/ha), beneficio (€/kg), agua (L/kg)
cultivos = {
    "maiz":    {"prod": 640, "ben": 1.0,  "agua": 8.75},
    "soja":    {"prod": 500, "ben": 0.8,  "agua": 5.0},
    "lechuga": {"prod": 400, "ben": 0.6,  "agua": 2.25},
}

# Crear modelo
model3 = LpProblem("Plantaciones", LpMaximize)

# Variables: hectáreas dedicadas a cada cultivo
x = {c: LpVariable(f"ha_{c}", lowBound=0) for c in cultivos}

# Función objetivo: maximizar beneficio
model3 += lpSum(x[c] * cultivos[c]["prod"] * cultivos[c]["ben"] for c in cultivos)

# Restricciones
# 1) Tierra total
model3 += lpSum(x[c] for c in cultivos) <= total_hect, "Tierra_disponible"
# 2) Agua total
model3 += lpSum(x[c] * cultivos[c]["prod"] * cultivos[c]["agua"] for c in cultivos) <= agua_disponible, "Agua_disponible"
# 3) Maíz mínimo
model3 += x["maiz"] * cultivos["maiz"]["prod"] >= min_maiz_kg, "Min_maiz"

# Resolver
model3.solve()
print("=== Problema 3: Plantaciones ===")
print("Status:", LpStatus[model3.status])
for c in cultivos:
    print(f"Hectáreas de {c}: {x[c].value():.2f}")
print("Beneficio total: €", value(model3.objective))
print()


# ----------------------------
# Problema 6: Menú (corrección)
# ----------------------------

# Variables: gramos de cada comida en el menú
y = {c: LpVariable(f"g_{c}", lowBound=0, upBound=limites[c]) for c in datos}

# Crear modelo (minimizar grasa)
model6 = LpProblem("Menu_Hospital", LpMinimize)

# Función objetivo: minimizar grasa total
# Reemplazamos (y[c]/100) por (y[c] * 0.01)
model6 += lpSum((y[c] * 0.01) * datos[c]["grasa"] for c in datos)

# Restricciones de nutrientes mínimos:
for nut in ["proteinas", "hierro", "niacina", "tiamina", "vitC"]:
    # Igual aquí: (y[c]/100) → (y[c] * 0.01)
    model6 += lpSum((y[c] * 0.01) * datos[c][nut] for c in datos) >= req[nut], f"Min_{nut}"

# Resolver
model6.solve()
print("=== Problema 6: Menú ===")
print("Status:", LpStatus[model6.status])
for c in datos:
    print(f"Gramos de {c}: {y[c].value():.1f} g")
print("Grasa total en el menú:", value(model6.objective), "mg")


=== Problema 3: Plantaciones ===
Status: Optimal
Hectáreas de maiz: 8.00
Hectáreas de soja: 10.88
Hectáreas de lechuga: 31.12
Beneficio total: € 16940.0

=== Problema 6: Menú ===
Status: Optimal
Gramos de espagueti: 300.0 g
Gramos de pavo: 283.3 g
Gramos de escalope: 200.0 g
Gramos de espinacas: 100.0 g
Gramos de pastel: 66.7 g
Grasa total en el menú: 54799.999880999996 mg
